# 参考
- [NLP关键词提取必备：从TFIDF到KeyBert范式原理优缺点与开源实现](https://mp.weixin.qq.com/s/H2aRyF3tNZCCOxa4Oy2JMw)
- [盘点 KeyBert、TextRank 等九种主流关键词提取算法原理及 Python 代码实现](https://zhuanlan.zhihu.com/p/568271135)
- [NLP关键词提取方法总结及实现](https://blog.csdn.net/asialee_bird/article/details/96454544)
- 

# 基于词袋加权的TFIDF算法

In [ ]:
import jieba.analyse
text = "美国总统拜登当地时间26日又语出惊人。据路透社报道，拜登当天在波兰的演 讲中，称俄罗斯总统普京“不能继续掌权了”。\
    白宫官员随后出来解释，克里姆林宫同日 也作出回应，称“这不是由拜登决定的”。报道称，在华沙皇家城堡发表的讲话中，拜登在\
    谴责普京后表示“看在上帝的份上，这个人不能继续掌权了”。对此，路透社表示，该言论引发了华盛顿方面对局势升级的担忧，美国\
    一直避免直接对乌克兰进行军事干预，并明确表示不支持政权更迭。随后，一名白宫官员对拜登这番话进行解释，称拜登的言论并不\
    代表华盛顿政策的转变，其目的是让世界为乌克兰问题上的长期冲突做好准备。该官员称，拜登意为“不能 允许普京对其邻国或该地区\
    行使权力”，而不是在讨论普京在俄罗斯的权力或俄 罗斯的政权 更迭问题。同日，路透社称，被问及拜登这一言论时，俄罗斯总统新闻\
    秘书佩斯科夫作出回应，称“这不是由拜登决定的。俄罗斯总统是由俄罗斯人选举产生的”。此前 ，俄罗斯安全会议副主席梅德韦杰夫接受\
    俄媒采访时曾表示，俄特别军事行动既定目标包括实现乌克兰去军事化、去纳粹化、成为中立国家、不奉行反俄政策。俄开展特别军事行动\
    首先是因为其设定的目标未能通过外交方式实现。他表示，社会调查显示，四分之三的俄罗斯民众支持在乌开展特别军事行动。西方国家企\
    图通过制裁影响俄民众，煽动他们反对政府，结果适得其反。"
jieba.analyse.extract_tags(text, topK=20, withWeight=True, allowPOS=('ns', 'n', 'vn', 'v','nr', 'nt'))

In [1]:
# 输入语料库
corpus = ['this is the first document',
        'this is the second second document',
        'and the third one',
        'is this the first document']

words_list = list()
for i in range(len(corpus)):
    words_list.append(corpus[i].split(' '))
### 第一种 gensim
from gensim import corpora
# 赋给语料库中每个词(不重复的词)一个整数id
dic = corpora.Dictionary(words_list)
# print(dic.token2id)
# 元组中第一个元素是词语在词典中对应的id，第二个元素是词语在文档中出现的次数
new_corpus = [dic.doc2bow(words) for words in words_list]

# 训练模型并保存
from gensim import models
tfidf = models.TfidfModel(new_corpus)
tfidf_vec = []
for i in range(len(corpus)):
    string = corpus[i]
    string_bow = dic.doc2bow(string.lower().split())
    string_tfidf = tfidf[string_bow]
    tfidf_vec.append(string_tfidf)
# 输出 词语id与词语tfidf值
print(tfidf_vec)

# ### 第二种 sklearn
# from sklearn.feature_extraction.text import TfidfVectorizer
# tfidf_vec = TfidfVectorizer()
# tfidf_matrix = tfidf_vec.fit_transform(corpus)
# # 得到语料库所有不重复的词
# print(tfidf_vec.get_feature_names())
# # 得到每个单词对应的id值
# print(tfidf_vec.vocabulary_)
# # 得到每个句子所对应的向量，向量里数字的顺序是按照词语的id顺序来的
# print(tfidf_matrix.toarray())

{'document': 0, 'first': 1, 'is': 2, 'the': 3, 'this': 4, 'second': 5, 'and': 6, 'one': 7, 'third': 8}
[[(0, 0.33699829595119235), (1, 0.8119707171924228), (2, 0.33699829595119235), (4, 0.33699829595119235)], [(0, 0.10212329019650272), (2, 0.10212329019650272), (4, 0.10212329019650272), (5, 0.9842319344536239)], [(6, 0.5773502691896258), (7, 0.5773502691896258), (8, 0.5773502691896258)], [(0, 0.33699829595119235), (1, 0.8119707171924228), (2, 0.33699829595119235), (4, 0.33699829595119235)]]


# 考虑词关联网络的TextRank算法

In [ ]:
import jieba.analyse
text = "美国总统拜登当地时间26日又语出惊人。据路透社报道，拜登当天在波兰的演 讲中，称俄罗斯总统普京“不能继续掌权了”。\
    白宫官员随后出来解释，克里姆林宫同日 也作出回应，称“这不是由拜登决定的”。报道称，在华沙皇家城堡发表的讲话中，拜登在\
    谴责普京后表示“看在上帝的份上，这个人不能继续掌权了”。对此，路透社表示，该言论引发了华盛顿方面对局势升级的担忧，美国\
    一直避免直接对乌克兰进行军事干预，并明确表示不支持政权更迭。随后，一名白宫官员对拜登这番话进行解释，称拜登的言论并不\
    代表华盛顿政策的转变，其目的是让世界为乌克兰问题上的长期冲突做好准备。该官员称，拜登意为“不能 允许普京对其邻国或该地区\
    行使权力”，而不是在讨论普京在俄罗斯的权力或俄 罗斯的政权 更迭问题。同日，路透社称，被问及拜登这一言论时，俄罗斯总统新闻\
    秘书佩斯科夫作出回应，称“这不是由拜登决定的。俄罗斯总统是由俄罗斯人选举产生的”。此前 ，俄罗斯安全会议副主席梅德韦杰夫接受\
    俄媒采访时曾表示，俄特别军事行动既定目标包括实现乌克兰去军事化、去纳粹化、成为中立国家、不奉行反俄政策。俄开展特别军事行动\
    首先是因为其设定的目标未能通过外交方式实现。他表示，社会调查显示，四分之三的俄罗斯民众支持在乌开展特别军事行动。西方国家企\
    图通过制裁影响俄民众，煽动他们反对政府，结果适得其反。"
jieba.analyse.textrank(text, topK=20, withWeight=True, allowPOS=('ns', 'n', 'vn', 'v','nr', 'nt'))

# 结合主题的LDA算法
`sklearn`库和`genism`库

In [ ]:
## 英文语料库
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
# Define the word list
word_list = [["running", "cars"],["better", "driving"]]

# Lemmatize list of words and join
def lemmatize_sentence(list):
    lemmatizer = WordNetLemmatizer()
    tagged_sentence = pos_tag(list)
    lemmatized_sentence = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in tagged_sentence]
    return lemmatized_sentence

# Lemmatize the word list
lemmatized_output = [lemmatize_sentence(word) for word in word_list]

In [ ]:
from gensim import corpora, models
from gensim.models import LdaModel

# Create the term dictionary of our corpus, where every unique term is assigned an index
dictionary = corpora.Dictionary(lemmatized_output)

# Convert list of documents (corpus) into Document Term Matrix using dictionary prepared above.
corpus = [dictionary.doc2bow(doc) for doc in lemmatized_output]

# Create a TF-IDF model from the corpus
tfidf = models.TfidfModel(corpus)

# Transform the corpus to TF-IDF
corpus_tfidf = tfidf[corpus]

In [ ]:
import matplotlib.pyplot as plt
from gensim.models import CoherenceModel
from tqdm import tqdm
# Create empty lists to store coherence and perplexity values
coherence_values = []
perplexity_values = []
range_topics = range(1, 6)

# Loop through different numbers of topics
for num_topics in tqdm(range_topics):
    # Train LDA model with the given number of topics
    ldamodel = LdaModel(corpus_tfidf, num_topics=num_topics, id2word=dictionary, passes=50)
    # Calculate coherence score
    coherence_model = CoherenceModel(model=ldamodel, texts=lemmatized_output, dictionary=dictionary, coherence='c_v')
    coherence = coherence_model.get_coherence()
    # Calculate perplexity
    perplexity = ldamodel.log_perplexity(corpus_tfidf)
    # Append coherence and perplexity values to the lists
    coherence_values.append(coherence)
    perplexity_values.append(perplexity)

# Plot the relationship between number of topics and coherence values
fig, ax1 = plt.subplots()
line1, = ax1.plot(range_topics, coherence_values, label='Coherence', color='blue')
ax1.set_xlabel('Number of Topics')
ax1.set_ylabel('Coherence', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Create a second y-axis for perplexity values
ax2 = ax1.twinx()
line2, = ax2.plot(range_topics, perplexity_values, label='Perplexity', color='red')
ax2.set_ylabel('Perplexity', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Set title and legend
plt.title('Coherence and Perplexity vs. Number of Topics')

# Combine legends
lines = [line1, line2]
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc='upper right')

# Show the plot
plt.show()


# 结合语义编码的KeyBert算法

In [2]:
from keybert import KeyBERT
import jieba 

# model = KeyBERT('bert-base-chinese')
# text = "美国总统拜登当地时间26日又语出惊人。据路透社报道，拜登当天在波兰的演 讲中，称俄罗斯总统普京“不能继续掌权了”。\
# 白宫官员随后出来解释，克里姆林宫同日也作出回应，称“这不是由拜登决定的”。报道称，在华沙皇家城堡发表的讲话中，\
# 拜登在谴责普京后表示“看在上帝的份上，这个人不能继续掌权了”。对此，路透社表示，该言论引发了华盛顿方面对局势升级的担忧，\
# 美国一直避免直接对乌克兰进行军事干预，并明确表示不支持政权更迭。随后，一名白宫官员对拜登这番话进行解释，称拜登的言论并不代表华盛顿政策的转变，\
# 其目的是让世界为乌克兰问题上的长期冲突做好准备。该官员称，拜登意为“不能 允许普京对其邻国或该地区行使权力”，\
# 而不是在讨论普京在俄罗斯的权力或俄罗斯的政权 更迭问题。同日，路透社称，被问及拜登这一言论时，俄罗斯总统新闻秘书佩斯科夫作出回应，\
# 称“这不是由拜登决定的。俄罗斯总统是由俄罗斯人选举产生的”。此前，俄罗斯安全会议副主席梅德韦杰夫接受俄媒采访时曾表示，\
# 俄特别军事行动既定目标包括实现乌克兰去军事化、去纳粹化、成为中立国家、不奉行反俄政策。俄开展特别军事行动首先是因为其设定的目标未能通过外交方式实现。\
# 他表示，社会调查显示，四分之三的俄罗斯民众支持在乌开展特别军事行动。西方国家企图通过制裁影响俄民众，煽动他们反对政府，结果适得其反。"
# doc = " ".join(jieba.cut(text))

model = KeyBERT('bert-base-uncased')
doc = "I like it very much"
keywords = model.extract_keywords(doc, keyphrase_ngram_range=(1,2),  top_n=20)
keywords

No sentence-transformers model found with name C:\Users\Lenovo/.cache\torch\sentence_transformers\bert-base-uncased. Creating a new one with MEAN pooling.


[('like', 0.4824)]